In [ ]:

!pip install --upgrade google-cloud


In [ ]:
!pip install db_dtypes

In [ ]:
!pip install google-cloud-bigquery

!gcloud --version

In [ ]:
!gcloud auth list
!gcloud auth application-default login

In [23]:
project_name = ""

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import matplotlib.pyplot as plt
import db_dtypes as db_dtypes

client = bigquery.Client(project=project_name)

# List datasets in the specified project
datasets = list(client.list_datasets())

# Print the dataset names
for dataset in datasets:
    print(dataset.dataset_id)

# TODO(developer): Set table_id to the ID of the destination table.
# table_id = "your-project.your_dataset.your_table_name"
sql = f'SELECT * FROM `{project_name}.mimic3_v1_4.PATIENTS`'
query_job = client.query(sql)

# df = query_job.to_dataframe()
rows = query_job.result()
patients_df = pd.DataFrame([dict(row) for row in rows])

plt.figure(figsize=(8, 6))
patients_df['GENDER'].value_counts().plot(kind='bar')

plt.title('Gender Distribution in MIMIC-III Patient Dataset')
plt.xlabel('Gender')
plt.ylabel('Count')

plt.show()

In [ ]:
!pip install bitarray
from bitarray import bitarray
import hashlib
import random
import math
random.seed(0)

class BloomFilter(object):
    def __init__(self, size, hash_count):
        """
        size: size of bit array
        hash_count: number of hash functions to use
        """
        self.size = size
        self.hash_count = hash_count
        self.bit_array = bitarray(size)
        self.bit_array.setall(0)
        self.hash_param = []
        i=0
        while i<hash_count:
            a=random.randint(1,9999)
            b=random.randint(1,9999)
            p = self.generate_large_prime(30)
            self.hash_param.append((a,b,p))
            i+=1
            
    def generate_large_prime(self, bit_size):

        random_number = random.getrandbits(bit_size)
        
        while not self.is_prime(random_number):
            random_number = random.getrandbits(bit_size)
        return random_number

    def is_prime(self, number):
        if number % 2 == 0:
            return False


        for i in range(3, int(math.sqrt(number)) + 1, 2):
            if number % i == 0:
                return False

        return True
    def calculate_hash(self,item,hash_params):
        item_val = (hash_params[0]*item + hash_params[1])%hash_params[2]
        #print (item_val,hash_params[2])
        return item_val
    
    
    def add(self, item):
        """
        Add an item to the filter
        """
        for p in self.hash_param:
            self.bit_array[self.calculate_hash(item, p)% self.size] = 1
        
    def lookup(self, item):
        """
        Check for existence of an item in filter
        """

        for p in self.hash_param:
            if self.bit_array[self.calculate_hash(item, p)% self.size] == 0:
                return False
        
        return True

In [ ]:
# SQL query to join the admissions table with the diagnoses table
sql = f"""
SELECT
    a.SUBJECT_ID,
    a.HADM_ID,
    d.ICD9_CODE,
    a.ADMITTIME
FROM
    `{project_name}.mimic3_v1_4.ADMISSIONS` as a
LEFT JOIN
    `{project_name}.mimic3_v1_4.DIAGNOSES_ICD` as d
ON
    a.SUBJECT_ID = d.SUBJECT_ID AND a.HADM_ID = d.HADM_ID
"""
query_job = client.query(sql)
rows = query_job.result()

diagnoses_df = pd.DataFrame([dict(row) for row in rows])

diagnoses_df['DIAGNOSIS_FLAG'] = diagnoses_df['ICD9_CODE'].notnull()

# Left join because want to keep all admissions to the hospital, and add false to flag if no diagnosis was given

ground_truth_df = diagnoses_df[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'ICD9_CODE', 'DIAGNOSIS_FLAG']]

ground_truth_df['DIAGNOSIS_FLAG'].fillna(False, inplace=True)
print("Ground Truth DataFrame:", len(ground_truth_df))
print(ground_truth_df.head(5))

In [ ]:
# Count the occurrences of each ICD9_CODE
icd9_counts = diagnoses_df['ICD9_CODE'].value_counts()

plt.figure(figsize=(10, 6))
icd9_counts.head(20).plot(kind='bar')

plt.title('Top 20 Most Common ICD-9 Diagnoses')
plt.xlabel('ICD-9 Code')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')

plt.show()

In [ ]:
icd_d_sql = f'SELECT * FROM `{project_name}.mimic3_v1_4.D_ICD_DIAGNOSES`'
icd_df_job = client.query(icd_d_sql)
icd_df_rows = icd_df_job.result()
icd_df = pd.DataFrame([dict(row) for row in icd_df_rows])
icd_df = icd_df.dropna(how='all')

ground_truth_df = pd.merge(diagnoses_df, icd_df, on=['ICD9_CODE'], how='left')
print(ground_truth_df.head(5))

icd9_counts = ground_truth_df['SHORT_TITLE'].value_counts()

plt.figure(figsize=(10, 6))
icd9_counts.head(20).plot(kind='bar')

plt.title('Top 20 Most Common ICD-9 Diagnoses')
plt.xlabel('ICD-9 Code')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')

plt.show()

In [ ]:

patients_df = patients_df.dropna(how='all')
admissions_sql = f'SELECT * FROM `{project_name}.mimic3_v1_4.ADMISSIONS`'
admissions_query_job = client.query(admissions_sql)
admissions_rows = admissions_query_job.result()
admissions_df = pd.DataFrame([dict(row) for row in admissions_rows])
admissions_df = admissions_df.dropna(how='all')
print(admissions_df['SUBJECT_ID'].isin(patients_df['SUBJECT_ID']).all())  # True means filtering is unnecessary

In [ ]:
# Bloom Filter Testing

bloom_filter = BloomFilter(size=100000, hash_count=5)

for subject_id in patients_df['SUBJECT_ID']:
    bloom_filter.add(subject_id)

filtered_admissions_df = admissions_df[admissions_df['SUBJECT_ID'].apply(
    lambda x: bloom_filter.lookup(x))]

result_df = pd.merge(patients_df, filtered_admissions_df, on='SUBJECT_ID', how='inner')

print("Patients DataFrame length", len(patients_df))
print("Admissions DataFrame length", len(admissions_df))
print("Filtered DataFrame length", len(filtered_admissions_df))
print("Patient-Admissions DataFrame length", len(result_df))

In [ ]:
#Bloom Filter Testing with Diagnoses Table
diagnoses_sql = f"SELECT * FROM `{project_name}.mimic3_v1_4.DIAGNOSES_ICD`"
diagnoses = client.query(diagnoses_sql)
diagnoses_rows = diagnoses.result()
diagnoses_icd_df = pd.DataFrame([dict(row) for row in diagnoses_rows])
print(admissions_df['SUBJECT_ID'].isin(diagnoses_icd_df['SUBJECT_ID']).all())  # True means filtering is unnecessary
print(patients_df['SUBJECT_ID'].isin(diagnoses_icd_df['SUBJECT_ID']).all())  # True means filtering is unnecessary
diagnoses_icd_df = diagnoses_icd_df.dropna(how='all')

print("Diagnoses DataFrame length", len(diagnoses_icd_df))
print("Patient-Admissions DataFrame length", len(result_df))

filtered_admissions_df = diagnoses_icd_df[diagnoses_icd_df['SUBJECT_ID'].apply(
    lambda x: bloom_filter.lookup(x))]

print("Filtered Admissions DataFrame length", len(filtered_admissions_df))
result_df = pd.merge(patients_df, diagnoses_icd_df, on='SUBJECT_ID', how='inner')

print("Patient-Admissions-Diagnoses DataFrame length", len(result_df))

In [32]:
labeevents_sql = f"""
SELECT SUBJECT_ID, HADM_ID, ITEMID, VALUE, VALUEUOM, FLAG  
FROM `{project_name}.mimic3_v1_4.LABEVENTS`  
WHERE ITEMID IN (50983, 50971, 50912, 51006, 50907, 50909, 50906, 50954, 51000, 50960, 50902, 50809, 50910, 50970, 50893, 51274, 51237, 50882)  
AND DATE(CHARTTIME) >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 YEAR)
AND RAND() <= 0.1;
"""
labeevents_job = client.query(labeevents_sql)
labeevents_rows = labeevents_job.result()
labeevents_df = pd.DataFrame([dict(row) for row in labeevents_rows])
labeevents_df = labeevents_df.dropna(how='all')

In [33]:
diagnoses_sql = f'SELECT * FROM `{project_name}.mimic3_v1_4.DIAGNOSES_ICD`'
diag_job = client.query(diagnoses_sql)
diag_rows = diag_job.result()
diag_df = pd.DataFrame([dict(row) for row in diag_rows])
diag_df = diag_df.dropna(how='all')

In [ ]:
#medical history experimentation
medical_history_df = diagnoses_df.merge(admissions_df[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME']], on=['SUBJECT_ID', 'HADM_ID'])
medical_history_df['ADMITTIME'] = pd.to_datetime(medical_history_df['ADMITTIME'])

#we want to only count the number of past diagnoses for each patient
medical_history_df = medical_history_df.groupby('SUBJECT_ID').apply(
    lambda group: group[group['ADMITTIME'] < group['ADMITTIME'].max()]
).reset_index(drop=True)

past_diagnoses_count = medical_history_df.groupby('SUBJECT_ID')['ICD9_CODE'].nunique().reset_index()
past_diagnoses_count.rename(columns={'ICD9_CODE': 'TOTAL_PAST_DIAGNOSES'}, inplace=True)

# if there were no previous diagnoses, change the NA to 0
patients_medical_df = pd.merge(result_df, past_diagnoses_count, on='SUBJECT_ID', how='left')
patients_medical_df['TOTAL_PAST_DIAGNOSES'].fillna(0, inplace=True) 

# add in lab events
feature_df = pd.merge(patients_medical_df, labeevents_df[['HADM_ID', 'ITEMID', 'VALUE']], 
                            on='HADM_ID', how='left')


#final joined dataset for feature analysis
feature_df.head()

In [ ]:
!pip install openai
import os
import openai
from openai import OpenAI

api_key_ai = os.getenv('OPENAI_API_KEY')
openai.api_key = api_key_ai

In [ ]:
import pandas as pd
import time

# Set your API key
openai.api_key = "Enter your OpenAI API key here"

# Simple function to create a prompt from patient data
def get_prompt(row):
    prompt = f"""
    Patient information:
    - Age: {row.get('AGE', 'Unknown')}
    - Gender: {row.get('GENDER', 'Unknown')}
    - Past diagnoses: {int(row['TOTAL_PAST_DIAGNOSES'])}
    - Death Flag: {'Yes' if row.get('DOD', None) is not None else 'No'}
    
    Lab results:
    """
    
    # can use itemid if that is provided
    if 'ITEMID' in row and 'VALUE' in row:
        prompt += f"- Lab {row['ITEMID']}: {row['VALUE']}\n"
    
    prompt += "\nWhat is the most likely diagnosis? Include ICD9 code."
    return prompt

def get_diagnosis(prompt):
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a medical expert. Provide diagnoses with ICD9 codes."},
            {"role": "user", "content": prompt}
        ],
    )
    return response.choices[0].message.content

# Sample a few rows to test
sample = feature_df.sample(3)
results = []

for _, row in sample.iterrows():
    prompt = get_prompt(row)
    
    diagnosis = get_diagnosis(prompt)
    results.append({
        'HADM_ID': row.get('HADM_ID', 'Unknown'),
        'predicted_diagnosis': diagnosis
    })

results_df = pd.DataFrame(results)
print(results_df)

In [27]:
pd.set_option('display.max_colwidth', None)
print(results_df['predicted_diagnosis'])

In [ ]:
patient_id = 249

note_events_sql = f"""
SELECT SUBJECT_ID, HADM_ID, DESCRIPTION, ISERROR, TEXT, CHARTDATE
FROM (
  SELECT *, ROW_NUMBER() OVER (PARTITION BY SUBJECT_ID ORDER BY CHARTDATE DESC) AS rn
  FROM `{project_name}.mimic3_v1_4.NOTEEVENTS`
  WHERE SUBJECT_ID = {patient_id}
)
WHERE rn = 1
"""

single_note_job = client.query(note_events_sql)
single_note_rows = single_note_job.result()
single_note_df = pd.DataFrame([dict(row) for row in single_note_rows])
single_note_df = single_note_df.dropna(how='all')
print(single_note_df.head(5))
print(len(single_note_df))
print(single_note_df['TEXT'].iloc[0])

In [ ]:
note_events_sql = f"""
SELECT SUBJECT_ID, HADM_ID, DESCRIPTION, ISERROR, TEXT, CHARTDATE
FROM (
  SELECT *, ROW_NUMBER() OVER (PARTITION BY SUBJECT_ID ORDER BY CHARTDATE DESC) AS rn
  FROM `{project_name}.mimic3_v1_4.NOTEEVENTS`
  WHERE HADM_ID IS NOT NULL
)
WHERE rn = 1
"""

note_job = client.query(note_events_sql)
note_rows = note_job.result()
note_df = pd.DataFrame([dict(row) for row in note_rows])
note_df = note_df.dropna(how='all')
print(note_df.head(5))
print(len(note_df))
print(note_df['TEXT'].iloc[0])

In [ ]:
import csv

filename = 'patient_notes.csv'
header = ['patient_id', 'notes']
rows = note_df[['SUBJECT_ID', 'TEXT']].values.tolist()

with open(filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    csvwriter.writerow(header)
    csvwriter.writerows(rows)

print(f"Created CSV file: {filename}")

In [ ]:
print("Number of entries in CSV file", len(rows))

In [ ]:
#Checking that all subject ids from notes_df are also in the ground truth

ids_ground_truth = set(test_ground_truth['SUBJECT_ID'])
ids_notes = set(note_df['SUBJECT_ID'])

missing_subject_ids = ids_ground_truth - ids_notes

print("Missing Patients", missing_subject_ids, len(missing_subject_ids))


In [ ]:
import csv 

filename = 'ground_truth.csv'
header = ['patient_id', 'diagnosis']
rows = test_ground_truth[['SUBJECT_ID', 'SHORT_TITLE']].values.tolist()

with open(filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    csvwriter.writerow(header)
    csvwriter.writerows(rows)

print(f"Created CSV file: {filename}")

Created CSV file: ground_truth.csv


In [30]:
import csv 

filename = 'primary_diagnosis.csv'
header = ['patient_id', 'diagnosis']
rows = one_diagnosis[['SUBJECT_ID', 'PRIMARY_DIAGNOSIS']].values.tolist()

with open(filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    csvwriter.writerow(header)
    csvwriter.writerows(rows)

print(f"Created CSV file: {filename}")

Created CSV file: primary_diagnosis.csv
